In [7]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("alexteboul/diabetes-health-indicators-dataset")

print("Path to dataset files:", path)

100%|██████████| 6.03M/6.03M [00:05<00:00, 1.10MB/s]

Extracting files...


Path to dataset files: /Users/amauryjanin/.cache/kagglehub/datasets/alexteboul/diabetes-health-indicators-dataset/versions/1
